In [153]:
import pandas as pd
import matplotlib

In [154]:
data = pd.read_csv('data/train_data/train_task_3_4.csv')
print(len(data))
data.head()

1382727


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,898,2111,280203,1,2,2
1,767,3062,55638,1,3,3
2,165,1156,386475,1,2,2
3,490,1653,997498,1,4,4
4,298,3912,578636,1,3,3


In [155]:
answer_metadata = pd.read_csv('data/metadata/answer_metadata_task_3_4.csv')
data=data.merge(answer_metadata, on="AnswerId", how="left")
data=data.groupby("QuestionId")[["Confidence","IsCorrect"]].mean()

# Create avg confidence
data=data.dropna(subset=["Confidence","IsCorrect"])

data

,Confidence,IsCorrect
QuestionId,,
0,66.176471,0.443457
2,66.201354,0.385214
3,85.021246,0.808757
4,59.343434,0.401408
5,77.419355,0.190876
...,...,...
943,83.793687,0.566528
944,69.852941,0.142857
945,72.244733,0.422336


In [156]:

# data=data.sort_values("Confidence",ascending=False).reset_index()

# data=data.sort_values("IsCorrect",ascending=True).reset_index()

import numpy as np
data["Quality"] = -np.abs(np.log(data["Confidence"]/(1-data["IsCorrect"])))
print(data)
data=data.sort_values("Quality",ascending=False).reset_index()

data


            Confidence  IsCorrect   Quality
QuestionId                                 
0            66.176471   0.443457 -4.778335
2            66.201354   0.385214 -4.679182
3            85.021246   0.808757 -6.097114
4            59.343434   0.401408 -4.596517
5            77.419355   0.190876 -4.561040
...                ...        ...       ...
943          83.793687   0.566528 -5.264287
944          69.852941   0.142857 -4.400543
945          72.244733   0.422336 -4.828823
946          72.058824   0.459459 -4.892668
947          75.240385   0.617124 -5.280733

[904 rows x 3 columns]


,QuestionId,Confidence,IsCorrect,Quality
0,548,25.000000,0.400000,-3.729701
1,169,47.910373,0.261946,-4.173070
2,775,45.000000,0.325000,-4.199705
3,442,50.000000,0.250000,-4.199705
4,273,47.286822,0.292683,-4.202508
...,...,...,...,...
899,312,87.255814,0.859442,-6.430980
900,828,90.748393,0.873935,-6.579046
901,825,89.021916,0.888384,-6.681570
902,422,92.968750,0.895833,-6.794027


In [157]:
eval_validation = pd.read_csv('data/test_data/quality_response_remapped_public.csv')
print(len(eval_validation))
eval_validation.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,0,0,909,318,2,2,2,1.0,2
1,16,16,285,609,2,2,2,2.0,2
2,48,48,132,271,1,2,2,2.0,1
3,38,38,704,127,2,2,1,2.0,2
4,33,33,16,586,1,1,2,2.0,2


In [158]:
eval_test = pd.read_csv('data/test_data/quality_response_remapped_private.csv')
print(len(eval_test))
eval_test.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,15,15,145,762,2,2,2,2.0,2
1,31,31,307,208,1,1,1,2.0,1
2,25,25,278,195,1,1,1,1.0,1
3,3,3,13,153,2,2,2,2.0,2
4,40,40,23,100,2,1,1,1.0,1


In [159]:
submission = pd.read_csv('submission/template.csv')
print(len(submission))
print(submission.head())
for idx in range(len(data)):
    tmp = data[data['QuestionId']==idx].index
    submission.loc[idx,'ranking'] = 987654321+idx if tmp.empty else tmp[0]
print(data[710:720])
submission.head()

948
   QuestionId  ranking
0           0      NaN
1           1      NaN
2           2      NaN
3           3      NaN
4           4      NaN
     QuestionId  Confidence  IsCorrect   Quality
710         798   68.046133   0.692612 -5.399831
711         823   79.140625   0.644354 -5.405045
712         930   91.666667   0.588235 -5.405462
713         309   85.526316   0.617647 -5.410235
714         739   82.500000   0.632184 -5.412971
715         314   78.369705   0.650667 -5.413166
716         403   78.012685   0.652292 -5.413265
717         837   84.375000   0.624806 -5.415584
718          20   82.482993   0.633861 -5.417333
719         659   77.036462   0.658960 -5.420033


,QuestionId,ranking
0,0,264.0
1,1,987654322.0
2,2,194.0
3,3,878.0
4,4,133.0


In [160]:
def order2preference(ord, QL, QR):
    preference = []
    for idx in range(len(QL)):
        rankL = ord[ord.QuestionId==QL[idx]].ranking.values[0]
        rankR = ord[ord.QuestionId==QR[idx]].ranking.values[0]
        preference.append(1 if rankL<rankR else 2)
    return preference

def evaluate(truth, submission):
    QL = list(truth.left)
    QR = list(truth.right)
    if len(QL) != len(QR):
        message = 'left and right lengths are not the same'
        exit(message)
    print(list(truth["preference"]))
    return sum(x==y for x,y in zip(truth["preference"],order2preference(submission,QL,QR)))/len(QL)

def evaluates(truth, submission):
    return [
        evaluate(truth.rename(columns={"T1_ALR":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T2_CL":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T3_GF":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T4_MQ":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T5_NS":"preference"})[["left","right","preference"]], submission)]


In [161]:
list_vali = evaluates(eval_validation,submission)
print(list(order2preference(submission,list(eval_validation.left),list(eval_validation.right))))
print(max(list_vali))
print(list_vali)

[2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1]
[2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1]
[2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1]
[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0]
[2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1]
[2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1]
0.84
[0.72, 0.76, 0.84, 0.64, 0.72]


In [162]:
list_test = evaluates(eval_test,submission)
print(max(list_test))
print(list_test)
print(list(order2preference(submission,list(eval_test.left),list(eval_test.right))))

[2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2]
[2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, nan, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0]
[2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2]
0.56
[0.52, 0.56, 0.4, 0.48, 0.52]
[2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2]
